# PyGrid Client Association Requests API

### Import Libs

In [1]:
from syft.grid.client.client import connect
from syft.grid.client.grid_connection import GridHTTPConnection
from syft.core.node.domain.client import DomainClient

import syft as sy
import torch as th

sy.VERBOSE = False

In [2]:
domain_client = connect(
    url="http://localhost:5000", # Domain Address
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
)

network_node = connect(
    url="http://localhost:5001", # Network Address
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
)

domain_client.setup(
    email="owner@myorg.com",
    password="owerpwd",
    domain_name="OpenMined Domain",
    token="9G9MJ06OQH",
)

In [3]:
network_node.users.create(
    email="owner@myorg.com", password="owerpwd"
)

### Connect performing authentication process on both nodes

In [4]:
domain_node = connect(
    url="http://localhost:5000", # Domain Address
    credentials={"email":"owner@myorg.com", "password":"owerpwd"},
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
)

network_node = connect(
    url="http://localhost:5001", # Network Address
    credentials={"email":"owner@myorg.com", "password":"owerpwd"},
    conn_type= GridHTTPConnection, # HTTP Connection Protocol
)

### Create an association request on Node1 to node2

In [5]:
domain_node.association_requests.create(
    name="OpenMined Network Association Request",
    address="http://localhost:5001",
    sender_address="http://localhost:5000")

### Check association requests on Node 1

In [6]:
domain_node.association_requests.all(pandas=True)

,id,date,name,address,sender_address,accepted,pending,handshake_value
0,1,2021-06-16 09:06:12.747511,OpenMined Network Association Request,http://localhost:5001,http://localhost:5000,False,True,c0d6e99927df2f8364996201a12a098d173c755e278286...


### Check association requests on Node 2

In [7]:
network_node.association_requests.all(pandas=True)

,id,date,name,address,sender_address,accepted,pending,handshake_value
0,1,2021-06-16 09:06:12.767212,OpenMined Network Association Request,http://localhost:5000,http://localhost:5001,False,True,c0d6e99927df2f8364996201a12a098d173c755e278286...


### Accept/Deny Association Request on Node 2

In [8]:
network_node.association_requests[1].accept()

{'msg': 'Association request replied!'}

In [9]:
domain_node.association_requests.all(pandas=True)

,id,date,name,address,sender_address,accepted,pending,handshake_value
0,1,2021-06-16 09:06:12.747511,OpenMined Network Association Request,http://localhost:5001,http://localhost:5000,True,False,c0d6e99927df2f8364996201a12a098d173c755e278286...


In [10]:
network_node.association_requests.all(pandas=True)

,id,date,name,address,sender_address,accepted,pending,handshake_value
0,1,2021-06-16 09:06:12.767212,OpenMined Network Association Request,http://localhost:5000,http://localhost:5001,True,False,c0d6e99927df2f8364996201a12a098d173c755e278286...


### Check if association request status were updated on both nodes

In [11]:
assert domain_node.association_requests[1].accepted == True
assert domain_node.association_requests[1].pending == False
assert network_node.association_requests[1].accepted == True
assert network_node.association_requests[1].pending == False

### Broadcast Search

Use this association to perform reach the domain by performing broadcast queries through the network

In [12]:
import torch as th

x = th.tensor([1,2,3,4,5,6,7,8,9])
x_s = x.send(domain_node, pointable=True, tags=['#diabetes', "x"])

In [13]:
network_node.search(query=["#diabetes"],pandas=True)

,node,items,address
0,OpenMined Domain,1,http://localhost:5000
